<b>Segmenting & Clustering Neighnorhoods in Toronto</b>

In [1]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [9]:
import requests as req

In [10]:
import bs4

In [11]:
from bs4 import BeautifulSoup as bs

In [12]:
import pandas as pd

Scrapping the <b>Wiki Page</b>:

In [13]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response= req.get(URL)
soup=bs(response.text, 'html.parser')
table= soup.find('table',{'class':'wikitable'}).tbody
#print(table)

In [14]:
rows=table.find_all('tr')
#print(rows)
columns=[v.text.replace('\n','') for v in rows[0].find_all('th')]
print(columns)

['Postcode', 'Borough', 'Neighbourhood']


In [15]:
df= pd.DataFrame(columns=columns)

In [16]:
for i in range(1, len(rows)):
    tds= rows[i].find_all('td')
    
    if len(tds)==2:
        values=[td[0].text, td[1].text, td[2].text.replace('\n', '').replace('\xa0','')]#this will fail
    else:
        values=[td.text.replace('\n', '').replace('\xa0','') for td in tds]
       #print(values)
        df= df.append(pd.Series(values,index=columns), ignore_index=True)

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [17]:
indexNames = df[ (df['Borough'] == 'Not assigned') ].index
df.drop(indexNames , inplace=True)

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [18]:
df=df.groupby("Postcode").agg(lambda x:','.join(set(x)))

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [19]:
df.loc[df['Neighbourhood']=="Not assigned",'Neighbourhood']=df.loc[df['Neighbourhood']=="Not assigned",'Borough']

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [20]:
df.shape

(103, 2)

In [21]:
df

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Port Union,Highland Creek,Rouge Hill"
M1E,Scarborough,"Guildwood,West Hill,Morningside"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
...,...,...
M9N,York,Weston
M9P,Etobicoke,Westmount
M9R,Etobicoke,"St. Phillips,Kingsview Village,Martin Grove Ga..."


In [22]:
pip install geocoder

  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491063 sha256=6e474b91ee4eefa6d572806b5112263b159703777d3d7fb9441c93b8a2f12ebf
  Stored in directory: C:\Users\Arpit\AppData\Local\pip\Cache\wheels\8b\99\a0\81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


Use the Geocoder package or the csv file to create the following dataframe:

In [23]:
URL="http://cocl.us/Geospatial_data"
df_Geo= pd.read_csv(URL)

In [24]:
df_Geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [25]:
df['Latitude']=df_Geo['Latitude'].values
df['Longitude']=df_Geo['Longitude'].values

In [26]:
df.head(50)

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
M1C,Scarborough,"Port Union,Highland Creek,Rouge Hill",43.784535,-79.160497
M1E,Scarborough,"Guildwood,West Hill,Morningside",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Oakridge,Golden Mile,Clairlea",43.711112,-79.284577
M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476


In [27]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [28]:
from geopy.geocoders import Nominatim

In [29]:
address = 'Toronto, TR'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

#Instead, let's chckout what's nearby our first Nearby from the dataset i.e;Malvern,Rouge



The geograpical coordinate of Toronto are 43.6561136, -79.392321.


In [31]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [32]:
import folium 

In [33]:
# create map of New York using latitude and longitude values

map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng in zip(df['Latitude'], df['Longitude']):
    label = '{}, {}'.format(lat, lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [34]:
CLIENT_ID = 'JHSQ3MWOYP22SMF0VAP4BEC5SO4PF1UNZXSFNW5C1BV3EGBG' # your Foursquare ID
CLIENT_SECRET = 'VVIRKROFORCBQ3Q2RWEUHJW0AU42WS5BBN4GUNTJP5SBDR1H' # your Foursquare Secret
VERSION = '20220303' # Foursquare API version
neighborhood_latitude=43.651070
neighborhood_longitude=-79.347015
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JHSQ3MWOYP22SMF0VAP4BEC5SO4PF1UNZXSFNW5C1BV3EGBG
CLIENT_SECRET:VVIRKROFORCBQ3Q2RWEUHJW0AU42WS5BBN4GUNTJP5SBDR1H


In [35]:
#url = 'https://api.foursquare.com/v2/venues/explore?&client_id=JHSQ3MWOYP22SMF0VAP4BEC5SO4PF1UNZXSFNW5C1BV3EGBG&client_secret=VVIRKROFORCBQ3Q2RWEUHJW0AU42WS5BBN4GUNTJP5SBDR1H&v=20220303&ll=43.651070,-79.347015&radius=5000&limit=50'

In [36]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=JHSQ3MWOYP22SMF0VAP4BEC5SO4PF1UNZXSFNW5C1BV3EGBG&client_secret=VVIRKROFORCBQ3Q2RWEUHJW0AU42WS5BBN4GUNTJP5SBDR1H&v=20220303&ll=43.65107,-79.347015&radius=1000&limit=100'

In [37]:
import json
from pandas.io.json import json_normalize

In [38]:
results = req.get(url).json()["response"]['groups'][0]['items']

#for i in range(2):
 #   awesome= results['response']['groups'][i]['items']

results

nearby_venues =json_normalize(results)
df_AWE=pd.DataFrame(nearby_venues)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [39]:
#df_AWE.head(50)

In [40]:
df_Fin= pd.DataFrame()

In [41]:
df_Fin['venue.location.lat'] = df_AWE['venue.location.lat']

In [42]:
df_Fin['venue.location.lng']=df_AWE['venue.location.lng']

In [43]:
df_Fin.head()

,venue.location.lat,venue.location.lng
0,43.647088,-79.351306
1,43.653756,-79.354390
2,43.644791,-79.351000
3,43.655764,-79.354806
4,43.655174,-79.340598


In [44]:
latitute=neighborhood_latitude
longitute=neighborhood_longitude

In [45]:
map_Fin = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng in zip(df_Fin['venue.location.lat'], df_Fin['venue.location.lng']):
    label = '{}, {}'.format(lat, lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Fin)  
    
map_Fin